In [1]:
import numpy as np
import sys
sys.path.append('../')
import general as gen
import astrometry as ast
from astropy.table import Table
from astropy import wcs
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord
from photutils.aperture import SkyCircularAperture, aperture_photometry

In [2]:
#load the RQ and RL catalogues
data_rq = Table.read(gen.PATH_CATS + 'RAGERS_COSMOS2020_matches_Mstar_z_rq.fits')
data_rl = Table.read(gen.PATH_CATS + 'RAGERS_COSMOS2020_matches_Mstar_z_rl.fits')

In [3]:
#load the S2COSMOS mask
mask_file = gen.PATH_DATA + 's2cosmos_main_mask.fits'
mask = fits.getdata(mask_file)[0]
w = wcs.WCS(fits.getheader(mask_file), naxis=2)

Set OBSGEO-B to    19.822880 from OBSGEO-[XYZ].
Set OBSGEO-H to     4120.022 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


In [4]:
#create a WCS object from the information in the header
#find the pixel scale (in arcsec/pix) in each dimension
ps_all = wcs.utils.proj_plane_pixel_scales(w) * 3600.
#use the average across all dimensions to estimate the scalar pixel scale of the image
ps = np.mean(ps_all)
print(ps)

1.9999999999999871


In [5]:
#get the positions of each source and use to create apertures
coords_rq = SkyCoord(data_rq['ALPHA_J2000'], data_rq['DELTA_J2000'])
apers_rq = [SkyCircularAperture(coords_rq, r=r*u.arcmin).to_pixel(w) for r in gen.r_search_all]
coords_rl = SkyCoord(data_rl['ALPHA_J2000'], data_rl['DELTA_J2000'])
apers_rl = [SkyCircularAperture(coords_rl, r=r*u.arcmin).to_pixel(w) for r in gen.r_search_all]


In [6]:
apphot_rq = aperture_photometry(mask, apers_rq, method='exact')	
apphot_rl = aperture_photometry(mask, apers_rl, method='exact')	

In [7]:
for i,r in enumerate(gen.r_search_all):
	data_rq[f'area_main_{int(r)}'] = apphot_rq[f'aperture_sum_{i}'] * (ps / 3600) ** 2
	data_rl[f'area_main_{int(r)}'] = apphot_rl[f'aperture_sum_{i}'] * (ps / 3600) ** 2

In [9]:
data_rq.write(gen.PATH_CATS + 'RAGERS_COSMOS2020_matches_Mstar_z_rq.fits', overwrite=True)
data_rl.write(gen.PATH_CATS + 'RAGERS_COSMOS2020_matches_Mstar_z_rl.fits', overwrite=True)